# This version scores the basic models:
- baseline (top ten)
- all words
- stemmed words
- lemmatized words

import files

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from operator import itemgetter
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

load

In [2]:
df = pd.read_csv('../data/df.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

could do more here to explore the neither and both values

move on to language processing

train-test split

In [3]:
X, y = df['text'].to_frame(), df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

add label to X_train for research purposes .. obviously don't include this in the model

reset index to anticipate future problems ... or not reset the index???

In [4]:
X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]

<ipython-input-4-0cb08c382a27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]


perfunctory exploring should happen here

top ten visualizations for pos. and non-pos.

size of vocabulary

more?

In [5]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

In [6]:
X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)
vocab_raw = set(X_train['text_tokenized'].explode())
print('Size of raw vocabulary:', len(vocab_raw))   

Size of raw vocabulary: 8876


<ipython-input-6-5ba9b3884ac5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)


In [7]:
def stringify(word_list):
    string = ''
    for word in word_list:
        string = string + word + ' '
    return string[:-1]

gonna need naive bayes, might not do any other models (markov, etc.)

In [8]:
baseline_model = MultinomialNB()

look at plurality winner to see score to beat

In [9]:
plurality_cv = round(y_train.value_counts(normalize=True)[0],4)
y_train.value_counts(normalize=True)

sentiment
0    0.672366
1    0.327634
Name: proportion, dtype: float64

first model, just ten features

In [10]:
tfidf = TfidfVectorizer(
    max_features = 10
)

X_train_vectorized = tfidf.fit_transform(X_train['text'])

baseline_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:', plurality_cv,
      '\nBaseline: ',baseline_cv)

Plurality: 0.6724 
Baseline:  0.6724


In [11]:
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [12]:
def stem_and_lemmatize_and_tokenize(document):
    tokens = tokenizer.tokenize(document)
    return [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]

In [13]:
X_train.loc[:, 'stem_lemma_list'] = X_train.loc[:, 'text'].apply(stem_and_lemmatize_and_tokenize)
vocab_stemmed_and_lemmatized = set(X_train['stem_lemma_list'].explode())
print('Size of raw vocabulary:                   ', len(vocab_raw),
      '\nSize of stemmed and lemmatized vocabulary:', len(vocab_stemmed_and_lemmatized)
     )

Size of raw vocabulary:                    8876 
Size of stemmed and lemmatized vocabulary: 6984


<ipython-input-13-f495710910f9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'stem_lemma_list'] = X_train.loc[:, 'text'].apply(stem_and_lemmatize_and_tokenize)


In [14]:
X_train['text_stemmed_and_lemmatized'] = X_train['stem_lemma_list'].apply(lambda x: stringify(x))

<ipython-input-14-91f45a5a7bce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['text_stemmed_and_lemmatized'] = X_train['stem_lemma_list'].apply(lambda x: stringify(x))


In [15]:
tfidf = TfidfVectorizer()

X_train_vectorized = tfidf.fit_transform(X_train['text_stemmed_and_lemmatized'])

stemmed_and_lemmatized_words_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:                   ', plurality_cv,
      '\nStemmed and Lemmatized Words:', stemmed_and_lemmatized_words_cv
     )

Plurality:                    0.6724 
Stemmed and Lemmatized Words: 0.6998


In [16]:
tfidf = TfidfVectorizer(
    min_df = 2,
    max_df = .02
)

X_train_vectorized = tfidf.fit_transform(X_train['text_stemmed_and_lemmatized'])

stemmed_and_lemmatized_words_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:                   ', plurality_cv,
      '\nStemmed and Lemmatized Words:', stemmed_and_lemmatized_words_cv
     )

Plurality:                    0.6724 
Stemmed and Lemmatized Words: 0.7186
